# MIT IQuHack - IonQ Challenge

In [6]:
import json
import pickle
import os
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from typing import Dict, List
from IPython.display import clear_output

import qiskit
from qiskit import quantum_info
from qiskit.execute_function import execute
from qiskit import BasicAer
from qiskit import QuantumCircuit
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import ZFeatureMap
from qiskit.quantum_info import SparsePauliOp
from qiskit.utils import algorithm_globals
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit_machine_learning.neural_networks import EstimatorQNN

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# algorithm_globals.random_seed = 12345

ImportError: cannot import name 'EstimatorQNN' from 'qiskit_machine_learning.neural_networks' (C:\Users\PranavMaheshwari\AppData\Local\Programs\Python\Python310\lib\site-packages\qiskit_machine_learning\neural_networks\__init__.py)

## Part 1: Encoding and Decoding

In [7]:
def simulate(circuit: qiskit.QuantumCircuit) -> dict:
    """Simulate the circuit, give the state vector as the result."""
    
    backend = BasicAer.get_backend('statevector_simulator')
    job = execute(circuit, backend)
    result = job.result()
    state_vector = result.get_statevector()
    histogram = dict()
    for i in range(len(state_vector)):
        population = abs(state_vector[i]) ** 2
        if population > 1e-9:
            histogram[i] = population
    
    return histogram

def histogram_to_category(histogram):
    assert abs(sum(histogram.values())-1)<1e-8
    positive=0
    for key in histogram.keys():
        digits = bin(int(key))[2:].zfill(20)
        if digits[-1]=='0':
            positive+=histogram[key]
        
    return positive

def count_gates(circuit: qiskit.QuantumCircuit) -> Dict[int, int]:
    """Returns the number of gate operations with each number of qubits."""
    return Counter([len(gate[1]) for gate in circuit.data])

def image_mse(image1,image2):
    return mean_squared_error(image1, image2)

In [ ]:
def encode_qiskit(image):
    desired_state = np.zeros(1024)
    for i in range(len(sample_img)):
        desired_state[i] = sample_img[i]
    nrm = np.linalg.norm(desired_state)
    desired_state = desired_state/nrm
    qc = QuantumCircuit(10)
    qc.initialize(desired_state, [i for i in range(10)])
    #qc.decompose().decompose().decompose().decompose().decompose().draw()
    return qc

In [9]:
#load the actual hackthon data (fashion-mnist)
data_path='data'

images=np.load(data_path+'/images.npy')
labels=np.load(data_path+'/labels.npy')

plt.imshow(images[1])

FileNotFoundError: [Errno 2] No such file or directory: 'data/images.npy'

In [ ]:
sample_img = np.matrix(images[10])
def downsizing(sample_img):
    test_img = np.zeros((4,4))
    col_break_pt = [5,12,19,26]
    row_break_pt = [5,12,19,26]
    for i in range(4):
        for j in range(4):
            if j==0:
                test_img[i,j] = round(np.mean(sample_img[9*i:row_break_pt[i],:col_break_pt[j]]),4)
            else:
                test_img[i,j] = round(np.mean(sample_img[9*i:row_break_pt[i],col_break_pt[j-1]:col_break_pt[j]]),4)
    return test_img

In [ ]:
test_img = downsizing(sample_img)
plt.imshow(test_img)

In [ ]:
def reconstruction(test_img):
    img = np.zeros((28,28))
    min_img = 1e-4
    for i in range(9):
        for j in range(5):
            img[8-i,4-j] = abs(round(test_img[0,0] - j**3*1e-5 - 3*i*1e-6 - 2*min_img,2))

    for i in range(9):
        for j in range(5):
            img[8-i,23+j] = abs(round(test_img[0,4]-2*min_img -j**3*1e-5 -i*3*1e-6,2))

    for k in range(3):
        for i in range(9):
            for j in range(6):
                img[i,5+k*6 +j]= round(test_img[0,k+1]-5*(abs(k-1))*min_img+j*1e-6  +i*1e-6,5-2*abs(k-1))

    for i in range(9):
        for j in range(5):
            img[17-i,4-j] = abs(round(test_img[1,0] - j**3*1e-5 - 2*min_img,2))
    for i in range(9):
        for j in range(5):
            img[17-i,23+j] = abs(round(test_img[1,4]-2*min_img -j**3*1e-5 ,2))
    for k in range(3):
        for i in range(9):
            for j in range(6):
                img[9+i,5+k*6 +j]= round(test_img[1,k+1]-5*(abs(k-1))*min_img,5-2*abs(k-1))

    for i in range(10):
        for j in range(5):
            img[18+i,5-j] = abs(round(test_img[2,0] - j**3*1e-5 -i**3*1e-6  - 2*min_img,2))
    for i in range(10):
        for j in range(5):
            img[18+i,27-j] = abs(round(test_img[2,4]-2*min_img-i**3*1e-6  -j**3*1e-5 ,2))
    for k in range(3):
        for i in range(10):
            for j in range(6):
                img[18+i,5+k*6 +j]= round(test_img[2,k+1]-4*(abs(k-1))*min_img-i**3*1e-6 ,5)
    return img
plt.imshow(reconstruction(test_img))

In [ ]:
np.linalg.norm(reconstruction(test_img)-sample_img)

In [ ]:
def decode(histogram): 
    n=15
    options = {}
    hist_keys = list(histogram.keys())
    for i in range(n):
        options[i]=0
    for i in range(len(histogram)):
        for j in range(i,len(histogram)):
            x = hist_keys[i]^hist_keys[j]
            if x and (not(x & (x-1))):
                val1 = bin(hist_keys[i])[2:].zfill(n)
                val2 = bin(hist_keys[j])[2:].zfill(n)
                for k in range(n):
                    if val1[k]!=val2[k]:
                        if options[n-1-k] ==0:                            
                            options[n-1-k] = 2*np.arctan(np.sqrt(histogram[hist_keys[j]]/histogram[hist_keys[i]]))
            else:
                continue
    image = []
    for i in range(n):
        image.append(round(options[i]/10,4))
    return reconstruction(np.asarray(image).reshape((3,5)))

In [ ]:
def encode_qiskit(image):
    img_arr = np.asarray(downsizing(image)).reshape(-1)
    desired_state = img_arr
    q = qiskit.QuantumRegister(len(desired_state))
    circuit = qiskit.QuantumCircuit(q)
    i=0
    for ele in desired_state:
        if ele>1e-8:
            circuit.rx(10*ele,i)
        i+=1
    circuit.decompose().draw()
    return circuit

In [ ]:
J = sample_img
encode_qiskit(J).draw()


In [ ]:
circuit=encode_qiskit(J)

#simulate circuit
histogram=simulate(circuit)

In [ ]:
n = len(images)
mse=0
gatecount=0

for image in images:
    #encode image into circuit
    #J = downsizing(image)
    circuit=encode_qiskit(image)

    #simulate circuit
    histogram=simulate(circuit)

    #count the number of 2-qubit gates
    gatecount+=count_gates(circuit)[2]
    #print(gatecount)

    #reconstruct the image
    image_re=decode(histogram)
    #image_rec = reconstruction(downsizing(image))
    #calculate mse
    mse+=image_mse(np.asarray(image),image_re)
#fidelity of reconstruction
f=1-mse/n
gatecount=gatecount/n

#score for part1
print(f*(0.999**gatecount))

## Part 2: Classification

In [ ]:
def generate_dataset(num_images):
    images = []
    labels = []
    # hor_array = np.zeros((6, 8))
    # ver_array = np.zeros((4, 8))
    
    hor_array = np.zeros((12, 16))
    ver_array = np.zeros((8, 16))

    j = 0
    for i in range(0, 13):
        if i != 3:
            hor_array[j][i] = np.pi / 2
            hor_array[j][i + 1] = np.pi / 2
            j += 1

    j = 0
    for i in range(0, 8):
        ver_array[j][i] = np.pi / 2
        ver_array[j][i + 8] = np.pi / 2
        j += 1

    for n in range(num_images):
        rng = algorithm_globals.random.integers(0, 2)
        if rng == 0:
            labels.append(-1)
            random_image = algorithm_globals.random.integers(0, 12)
            images.append(np.array(hor_array[random_image]))
        elif rng == 1:
            labels.append(1)
            random_image = algorithm_globals.random.integers(0, 8)
            images.append(np.array(ver_array[random_image]))

        # Create noise
        for i in range(16):
            if images[-1][i] == 0:
                images[-1][i] = algorithm_globals.random.uniform(0, np.pi / 4)
    return images, labels

In [ ]:
images, labels = generate_dataset(50)

train_images, test_images, train_labels, test_labels = train_test_split(
    images, labels, test_size=0.3
)

#Let’s see some examples in our dataset

fig, ax = plt.subplots(2, 2, figsize=(10, 6), subplot_kw={"xticks": [], "yticks": []})
for i in range(4):
    ax[i // 2, i % 2].imshow(
        train_images[i].reshape(4, 4),  # Change back to 2 by 4
        aspect="equal",
    )
plt.subplots_adjust(wspace=0.1, hspace=0.025)


In [ ]:
# We now define a two qubit unitary as defined in [3]
def conv_circuit(params):
    target = QuantumCircuit(2)
    target.rz(-np.pi / 2, 1)
    target.cx(1, 0)
    target.rz(params[0], 0)
    target.ry(params[1], 1)
    target.cx(0, 1)
    target.ry(params[2], 1)
    target.cx(1, 0)
    target.rz(np.pi / 2, 0)
    return target


# Let's draw this circuit and see what it looks like
params = ParameterVector("θ", length=3)
circuit = conv_circuit(params)
circuit.draw("mpl")

In [ ]:
def conv_layer(num_qubits, param_prefix):
    qc = QuantumCircuit(num_qubits, name="Convolutional Layer")
    qubits = list(range(num_qubits))
    param_index = 0
    params = ParameterVector(param_prefix, length=num_qubits * 3)
    for q1, q2 in zip(qubits[0::2], qubits[1::2]):
        qc = qc.compose(conv_circuit(params[param_index : (param_index + 3)]), [q1, q2])
        qc.barrier()
        param_index += 3
    for q1, q2 in zip(qubits[1::2], qubits[2::2] + [0]):
        qc = qc.compose(conv_circuit(params[param_index : (param_index + 3)]), [q1, q2])
        qc.barrier()
        param_index += 3

    qc_inst = qc.to_instruction()

    qc = QuantumCircuit(num_qubits)
    qc.append(qc_inst, qubits)
    return qc


circuit = conv_layer(16, "θ")
circuit.decompose().draw("mpl")

In [ ]:
def pool_circuit(params):
    target = QuantumCircuit(2)
    target.rz(-np.pi / 2, 1)
    target.cx(1, 0)
    target.rz(params[0], 0)
    target.ry(params[1], 1)
    target.cx(0, 1)
    target.ry(params[2], 1)

    return target


params = ParameterVector("θ", length=3)
circuit = pool_circuit(params)
circuit.draw("mpl")

In [ ]:
def pool_layer(sources, sinks, param_prefix):
    num_qubits = len(sources) + len(sinks)
    qc = QuantumCircuit(num_qubits, name="Pooling Layer")
    param_index = 0
    params = ParameterVector(param_prefix, length=num_qubits // 2 * 3)
    for source, sink in zip(sources, sinks):
        qc = qc.compose(pool_circuit(params[param_index : (param_index + 3)]), [source, sink])
        qc.barrier()
        param_index += 3

    qc_inst = qc.to_instruction()

    qc = QuantumCircuit(num_qubits)
    qc.append(qc_inst, range(num_qubits))
    return qc


sources = [i for i in range(0,8)]
sinks = [i for i in range(8,16)]
circuit = pool_layer(sources, sinks, "θ")
circuit.decompose().draw("mpl")


In [ ]:
feature_map = ZFeatureMap(16)
feature_map.decompose().draw("mpl")

In [ ]:
feature_map = ZFeatureMap(16)

ansatz = QuantumCircuit(16, name="Ansatz")

# First Convolutional Layer
ansatz.compose(conv_layer(16, "с1"), list(range(16)), inplace=True)

# First Pooling Layer
ansatz.compose(pool_layer([i for i in range(0,8)], [i for i in range(8,16)], "p1"), list(range(16)), inplace=True)

# First Convolutional Layer
ansatz.compose(conv_layer(8, "с2"), list(range(8,16)), inplace=True)

# First Pooling Layer
ansatz.compose(pool_layer([i for i in range(0,4)], [i for i in range(4,8)], "p2"), list(range(8,16)), inplace=True)

# Second Convolutional Layer
ansatz.compose(conv_layer(4, "c3"), list(range(12, 16)), inplace=True)

# Second Pooling Layer
ansatz.compose(pool_layer([i for i in range(0,2)], [i for i in range(2,4)], "p3"), list(range(12,16)), inplace=True)

# Third Convolutional Layer
ansatz.compose(conv_layer(2, "c4"), list(range(14, 16)), inplace=True)

# Third Pooling Layer
ansatz.compose(pool_layer([0], [1], "p4"), list(range(14,16)), inplace=True)

# Combining the feature map and ansatz
circuit = QuantumCircuit(16)
circuit.compose(feature_map, range(16), inplace=True)
circuit.compose(ansatz, range(16), inplace=True)

observable = SparsePauliOp.from_list([("Z" + "I" * 15, 1)])

# we decompose the circuit for the QNN to avoid additional data copying
qnn = EstimatorQNN(
    circuit=circuit.decompose(),
    observables=observable,
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
)

In [ ]:
circuit.draw("mpl")

In [ ]:
def callback_graph(weights, obj_func_eval):
    clear_output(wait=True)
    objective_func_vals.append(obj_func_eval)
    plt.title("Objective function value against iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Objective function value")
    plt.plot(range(len(objective_func_vals)), objective_func_vals)
    plt.show()

In [ ]:
classifier = NeuralNetworkClassifier(
    qnn,
    optimizer=COBYLA(maxiter=200),  # Set max iterations here
    callback=callback_graph,
    #initial_point=initial_point,
)

In [ ]:
x = np.asarray(train_images)
y = np.asarray(train_labels)

objective_func_vals = []
plt.rcParams["figure.figsize"] = (12, 6)
classifier.fit(x, y)

# score classifier
print(f"Accuracy from the train data : {np.round(100 * classifier.score(x, y), 2)}%")

In [ ]:
y_predict = classifier.predict(test_images)
x = np.asarray(test_images)
y = np.asarray(test_labels)
print(f"Accuracy from the test data : {np.round(100 * classifier.score(x, y), 2)}%")

# Let's see some examples in our dataset
fig, ax = plt.subplots(2, 2, figsize=(10, 6), subplot_kw={"xticks": [], "yticks": []})
for i in range(0, 4):
    ax[i // 2, i % 2].imshow(test_images[i].reshape(4, 4), aspect="equal")
    if y_predict[i] == -1:
        ax[i // 2, i % 2].set_title("The QCNN predicts this is a Horizontal Line")
    if y_predict[i] == +1:
        ax[i // 2, i % 2].set_title("The QCNN predicts this is a Vertical Line")
plt.subplots_adjust(wspace=0.1, hspace=0.5)